'''
v0: 국내 통합코드 적용
v1: title 전처리
v2: name 전처리
v3: 통합 안되는 친구들 필터 테스트
'''

In [47]:
from pymongo import MongoClient
import itertools
import numpy as np
import jaro
import time
#import multicpu

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근
filters_category = client['PUBLIC']['FilterCategory']

f_id = 0 #input
keyid = 800 #keyid

fid_key_query = filter_info.find_one({ '$and': [{ 'fId': f_id }, { 'keyId': keyid }]}) #f_id serach

pinst = []
pyear = []
pjournal = []
plang = []

if  fid_key_query != None: #f_id check
    for key in fid_key_query.keys() :
        if key == 'pFilter':
            pinst = fid_key_query[key]['inst']
            pyear = fid_key_query[key]['year']
            pjournal = fid_key_query[key]['journal']
            plang = fid_key_query[key]['lang']

wos_raw = client['WOS']['Rawdata']
scopus_raw = client['SCOPUS']['Rawdata']

wos_key_query = wos_raw.find({ 'keyId' : keyid })
scopus_key_query = scopus_raw.find({ 'keyId' : keyid })

key_querys = [wos_key_query, scopus_key_query] #Rawdata
id_domestic = client['ID']['Domestic'] #Domestic

mng_id = [] # Author id
paper = []

Answer_dict = {} # Answer result
fp_dict = {} #filter papaer result
site = ['WOS', 'SCOPUS']

savetime1 = 0
savetime2 = 0
end1 = 0

f_pyear = {}
f_pinst = {}
f_pjournal = {}
f_plang = {}

Inte_name = []

def simple_filter(value, filters):
    if value in filters or filters == []:
        return True
    return False
        
def fc_simple_filter(category, fc_dict):
    if category not in fc_dict:
        fc_dict[category] = 0
    fc_dict[category] += 1
    return fc_dict

for i in range(len(key_querys)):
    mng_dict = {}
    start1 = time.time()
    for key_query in key_querys[i]: #rawdata(magid, paper) insert

        if key_query['author_inst'] not in "":
            paper_year =  str(key_query['issue_year'])
            paper_journal = key_query['journal']
            # ori_inst = key_query['originalName'].split(';')[-2]
            ori_inst = key_query['author_inst'].split(';')[-2]
            paper_lang = key_query['issue_lang']
            exi_inst = key_query['author_inst'].split(';')[-2]
            mng_name = key_query['author'].split(';')[-2]
            mng_id = key_query['author_id'].split(';')[-1]
            paper = key_query['_id']
            
        if simple_filter(paper_year, pyear) and simple_filter(paper_journal, pjournal) and simple_filter(ori_inst, pinst) and simple_filter(paper_lang, plang):
            if mng_id not in mng_dict:
                mng_dict[mng_id] = {'name' : mng_name, 'inst' : exi_inst, 'papers' : [], 'oriInst' : ori_inst}
            mng_dict[mng_id]['papers'].append(paper)
            fp_dict[paper] = {'year' : paper_year, 'inst' : ori_inst, 'journal' : paper_journal, 'lang' : paper_lang}
            
    end2 = time.time()
    db_time = end2-start1
    print(f'DB 수집: {site[i]}, {db_time}')
    savetime1 += db_time
    len_mng_dict = len(mng_dict)
    print(f'전체 저자 수: {len_mng_dict}')
    
    for mng_one in mng_dict :
        oriinst = mng_dict[mng_one]['oriInst']
        exiinst = mng_dict[mng_one]['inst']
        mng_name = mng_dict[mng_one]['name']
        paper = mng_dict[mng_one]['papers']
        pre_name = mng_name.lower().replace(" ", "").replace(",", "").replace(".", "").replace("-", "")
        Answer = {'fid': f_id, 'keyId': keyid, 'name' : mng_name, 'pre_name' : pre_name, 'inst': oriinst, site[i] : {'inst' :exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst} }
        
        if pre_name not in Answer_dict and pre_name+'_0' not in Answer_dict : #동명이인이 없을 때
            Answer_dict[pre_name] = Answer
        else:
            count = 0
            flag = True
            while flag :
                temp = None
                tempName = pre_name
                
                if tempName in Answer_dict : # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = pre_name+'_'+str(count) # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                        
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' and key != 'keyId' and key != 'fid' and key != 'inst' and key != 'pre_name': 
                        src = ""
                        tgt = ""

                        if len(exiinst) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = exiinst

                        elif len(exiinst) < len(temp[key]['inst']):
                            src = exiinst
                            tgt = temp[key]['inst']

                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == exiinst or (src != "" and src in tgt) :  # 소속 같을때
                                Inte_name.append(tempName)
                                Answer_dict[tempName][site[i]]['A_id'].extend([mng_one])
                                Answer_dict[tempName][site[i]]['papers'].extend(paper)
                                flag = False
                                break

                            elif pre_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[pre_name+'_'+str(count+1)] = Answer
                                if tempName == pre_name:
                                    Answer_dict[pre_name+'_0'] = temp
                                    del Answer_dict[pre_name]
                                flag = False
                                break
                            
                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == exiinst  or (src != "" and src in tgt):  # 소속 같을때
                                Answer_dict[tempName][site[i]] =  {'inst' : exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst}
                                Inte_name.append(tempName)
                                if '대학교' in Answer_dict[tempName][site[i]]['oriInst'] and '대학교' not in Answer_dict[tempName]['inst']:
                                    Answer_dict[tempName]['inst'] = Answer_dict[tempName][site[i]]['oriInst']
                                flag = False
                                break
                            
                            elif pre_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[pre_name+'_'+str(count+1)] = Answer
                                if tempName == pre_name:
                                    Answer_dict[pre_name+'_0'] = temp
                                    del Answer_dict[pre_name]
                                flag = False
                                break

                count += 1
end3 = time.time()
savetime2 = end3-end2+savetime1

print(f'2차 통합: {savetime2}')
# print(sorted(Answer_dict.items()))
# print(filter_dict)
print(Inte_name)
print(len(Answer_dict))

DB 수집: WOS, 0.06434845924377441
전체 저자 수: 269
DB 수집: SCOPUS, 0.4404256343841553
전체 저자 수: 1123
2차 통합: 0.5067739486694336
[]
1392


In [48]:
def filter(rawdata):

    coauthor = rawdata['author'].split(";")[1:-1]
    year = int(rawdata['issue_year'])
    paper_keyword = rawdata['paper_keyword']
    
    if paper_keyword == [] or paper_keyword is None:
        keyword = []
    elif len(paper_keyword) > 1:
        for i in range(0, len(paper_keyword)):
            keyword = []
            keyword.append(paper_keyword[i].replace(" ", "").split("."))
    else:
        keyword = paper_keyword.replace(" ", "").split(".")

    journal = rawdata['journal']
    conference = rawdata['issue_inst']
    title = rawdata['title']

    return coauthor, year, keyword, journal, conference, title

def Secondary_filter(name, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst = 0
    weight = 0
    joc = 0
    coauthor1, year1, keyword1, journal1, conference1, title1 = filter(raw_one1)
    coauthor2, year2, keyword2, journal2, conference2, title2 = filter(raw_one2)

    if inst1 == inst2:
        inst = 1
    else:
        inst = jaro.jaro_winkler_metric(inst1, inst2)

    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])
    
    paper_sim = jaro.jaro_winkler_metric(title1.lower(), title2.lower())
                            
    if paper_sim >= 0.8 or inst >= 0.8:
        weight = 4
        # print(f'weigth: {weight} | joc: {joc} | yop: {year1}, {year2} | co_author_count: {co_author_count}| keyword: {keyword1}, {keyword1}')
        # print(f'name: {name} | inst1: {inst1} | title1: {title1} | site1: {site1}')
        # print(f'name: {name} | inst2: {inst2} | title2: {title2} | site1: {site2}')
        # print(f'inst: {inst} | journal1: {journal1} | journal2: {journal2}')
        # print('------------------------------------------------------------구분선2------------------------------------------------------------')
        return weight

    else:
        joc = 1 if journal1 == journal2 and conference1 == conference2 else 0

            
    yop = -(2*(abs(year1-year2)/20)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
        
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    weight = joc + yop + co_authorship + keyword
    
    print(f'weigth: {weight} | joc: {joc} | yop: {yop}, {year1}, {year2} | co_authorship: {co_authorship} | keyword: {keyword}')
    print(f'name: {name} | inst1: {inst1} | title1: {title1} | site1: {site1}')
    print(f'name: {name} | inst2: {inst2} | title2: {title2} | site1: {site2}')
    print(f'inst: {inst} | journal1: {journal1} | journal2: {journal2}')
    print('------------------------------------------------------------구분선------------------------------------------------------------')
    
    return weight

In [49]:
raw_dbs = {'WOS' : wos_raw, 'SCOPUS' : scopus_raw}
savetime1 = 0
savetime2 = 0
def getRaw(name):
    if 'raws' not in Answer_dict[name]:
        raws = []
        for site_one in site:
            if site_one in Answer_dict[name]:
                for c in raw_dbs[site_one].find({"_id": {"$in": Answer_dict[name][site_one]['papers']}}):
                    c['site'] = site_one
                    raws.append(c)
        
        Answer_dict[name]['raws'] = raws

processedList = []
deleteList = []

count_rule = 0
for Answer_one in Answer_dict :
    
    if '_' in Answer_one :
        start1 = time.time()
        name = Answer_one.split("_")
        if name[0] in processedList :
            continue
        preprocessedList = []
        c = 0
        while True :
            pname = name[0]+"_"+str(c)
            if pname in Answer_dict :            
                preprocessedList.append(pname)
                getRaw(pname)
                c += 1
            else :
                break
        end1 = time.time()
        savetime1 += end1 - start1
        processedList.append(name[0])
        flag = True
        while flag :
            flag = False
            pairs =list(itertools.combinations(preprocessedList, 2))
            
            for pair in pairs:
                pair = list(pair)

                raws1 = Answer_dict[pair[0]]['raws']
                raws2 = Answer_dict[pair[1]]['raws']
                
                for ra1, ra2 in zip(raws1, raws2):
                    site1 = ra1['site']
                    site2 = ra2['site']
                    inst1 = Answer_dict[pair[0]][site1]['oriInst']
                    inst2 = Answer_dict[pair[1]][site2]['oriInst']
                    count_rule += 1
                    if Secondary_filter(name[0], site1, inst1, ra1, site2, inst2, ra2) >= 3:
                        Inte_name.append(pair[0])
                        deleteList.append(pair[1])
                        for site_one in site:
                            if site_one in Answer_dict[pair[1]]:
                                if site_one in Answer_dict[pair[0]].keys() :                            
                                    Answer_dict[pair[0]][site_one]['A_id'].extend(Answer_dict[pair[1]][site_one]['A_id'])
                                    Answer_dict[pair[0]][site_one]['papers'].extend(Answer_dict[pair[1]][site_one]['papers'])
                                    Answer_dict[pair[0]]['raws'].extend(Answer_dict[pair[1]]['raws'])
                                    
                                    Answer_dict[pair[0]][site_one]['A_id'] = list(set(Answer_dict[pair[0]][site_one]['A_id']))
                                    Answer_dict[pair[0]][site_one]['papers'] = list(set(Answer_dict[pair[0]][site_one]['papers']))
                                else:
                                    Answer_dict[pair[0]][site_one] = Answer_dict[pair[1]][site_one]
   
                        flag = True
                        preprocessedList.remove(pair[1])
                        break
                if flag :
                    break
                
for d in deleteList:
    del Answer_dict[d]

for d in Answer_dict : 
    if 'raws' in Answer_dict[d] :
        del Answer_dict[d]['raws']
print(len(Inte_name))
print(count_rule)
# print(savetime1, savetime2)
# print(Answer_dict)
# id_domestic.insert_many(Answer_dict.values()) #mongodb 추가

weigth: 2.503736393798392 | joc: 1 | yop: 0.5, 2016, 2021 | co_authorship: 0.3716158349698344 | keyword: 0.6321205588285577
name: lixia | inst1: Harbin Med Univ, Dept Neurol, Affiliated Hosp 2, Harbin 150081, Peoples R China | title1: Lnc2Cancer: a manually curated database of experimentally supported lncRNAs associated with various human cancers | site1: WOS
name: lixia | inst2: Hainan Med Univ, Coll Biomed Informat & Engn, Hainan Women & Childrens Med Ctr, Key Lab Trop Translat Med,Minist Educ, Haikou 571199, Hainan, Peoples R China | title2: ImmReg: the regulon atlas of immune-related pathways across cancer types | site1: WOS
inst: 0.591753029147208 | journal1: NUCLEIC ACIDS RES | journal2: NUCLEIC ACIDS RES
------------------------------------------------------------구분선------------------------------------------------------------
weigth: 1.025106465816068 | joc: 0 | yop: 0.4, 2016, 2022 | co_authorship: 0.625106465816068 | keyword: 0.0
name: lixia | inst1: Harbin Med Univ, Dept Neur

In [50]:
import json

answer = None
numCor = 0
numErr = 0
numInteErr = 0
answerCounter = {}

def ansCheck(result, name) :
    global answer, numCor, numErr, answerCounter, numInteErr
    flag = True

    numK = len(result.keys())
    if numK-5 != len(answer[name].keys()) :
        flag = False
    else :
        for site in result.keys() :        
            if site != 'name' and site != 'fid' and site != 'keyId' and site != 'inst' and site != 'pre_name':
                if site not in answer[name].keys() :
                    flag = False
                else: 
                    if numK > 5:
                        if answer[name][site] != result[site]['inst'] :
                            flag = False
                    else :
                        if answer[name][site] != result[site]['inst'] and  answer[name][site] != result[site]['inst'].split(' ')[0]:
                            flag = False
    if flag :
        numCor += 1
        answerCounter[name] += 1
    else :
       
        if len(answer[name].keys()) > 1 :
            numInteErr += 1
            print(f"IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct \n - Answer : {answer[name]}\n - Result : {result}")
        else :
            pass
        numErr += 1
    return flag

with open(f'answer{keyid}.json', 'r',encoding='utf-8-sig') as a_json :

    answer = json.load(a_json)
    numAns = print(len(answer))
    for key in answer.keys() :
        answerCounter[key] = 0

    for answer_one in Answer_dict:
        test = answer_one
        
        name = Answer_dict[test]['name'].split('_')[0]

        if name in answer :
            ansCheck(Answer_dict[test], name)

        else :
            count = 0
            while name+str(count) in answer :
                if ansCheck(Answer_dict[test], name+str(count)) :
                    break
                count += 1

    print(f"num코렉트 {numCor}")
    print(f"numEror {numErr}")
    print(f"numInteEror {numInteErr}")
    totalError = 0
    totalInteError1 = 0
    totalInteError2 = 0
    answercount = 0
    totalInte = 0

    for key in answerCounter.keys() :
        answercount += 1
        if answerCounter[key] == 0 :
            totalError += 1
            print("answer=",key, answer[key])
            if len(answer[key].keys()) > 1 :
                totalInteError1 += 1

        else:
            if len(answer[key].keys()) > 1 :
                totalInteError2 += 1

    print("토탈앤써", answercount) 
    print("토탈인터그레이션", totalInteError1 + totalInteError2)
    print("--------------------------------")
    print("토탈에러", totalError) 
    print("토탈인터그레이션에러", totalInteError1)
    print(keyid)
    
print(sorted(Answer_dict.items()))

1240
IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct 
 - Answer : {'SCOPUS': 'Lawrence Berkeley Laboratory Berkeley CA United States', 'WOS': 'Lawrence Berkeley Lab, Berkeley, CA USA'}
 - Result : {'fid': 0, 'keyId': 800, 'name': 'Shoshani, Arie', 'pre_name': 'shoshaniarie', 'inst': 'Lawrence Berkeley Lab, Berkeley, CA USA', 'WOS': {'inst': 'Lawrence Berkeley Lab, Berkeley, CA USA', 'A_id': ['s6895024'], 'papers': [ObjectId('6268eca9b35bd9ffb2ddc7b6')], 'oriInst': 'Lawrence Berkeley Lab, Berkeley, CA USA'}}
IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct 
 - Answer : {'SCOPUS': 'FIVAN Valencia Spain', 'WOS': 'FIVAN, Valencia, Spain'}
 - Result : {'fid': 0, 'keyId': 800, 'name': 'Gagliardo, Pablo', 'pre_name': 'gagliardopablo', 'inst': 'FIVAN, Valencia, Spain', 'WOS': {'inst': 'FIVAN, Valencia, Spain', 'A_id': ['s6894923'], 'papers': [ObjectId('6268eca8b35bd9ffb2ddc72c')], 'oriInst': 'FIVAN, Valencia, Spain'}}
IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct 
 - Answer : {'SCOPUS': 'School of Mechanical Engi

In [40]:
def Secondary_filter2(name, name2, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst = 0
    weight = 0
    joc = 0
    coauthor1, year1, keyword1, journal1, conference1, title1 = filter(raw_one1)
    coauthor2, year2, keyword2, journal2, conference2, title2 = filter(raw_one2)

    if inst1 == inst2:
        inst = 1
    else:
        inst = jaro.jaro_winkler_metric(inst1, inst2)

    if name in coauthor1:
        coauthor1.remove(name)

    if name2 in coauthor2:
        coauthor2.remove(name2)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])
    
    paper_sim = jaro.jaro_winkler_metric(title1.lower(), title2.lower())
                            
    if paper_sim >= 0.8 or inst >= 0.8:
        weight = 4
        print(f'weigth: {weight} | joc: {joc} | yop: {year1}, {year2} | co_author_count: {co_author_count}| keyword: {keyword1}, {keyword1}')
        print(f'name: {name} | inst1: {inst1} | title1: {title1} | site1: {site1}')
        print(f'name: {name2} | inst2: {inst2} | title2: {title2} | site1: {site2}')
        print(f'inst: {inst} | journal1: {journal1} | journal2: {journal2}')
        print('------------------------------------------------------------구분선2------------------------------------------------------------')
        return weight

    else:
        joc = 1 if journal1 == journal2 and conference1 == conference2 else 0

            
    yop = -(2*(abs(year1-year2)/20)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
        
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    weight = joc + yop + co_authorship + keyword
    
    print(f'weigth: {weight} | joc: {joc} | yop: {yop}, {year1}, {year2} | co_authorship: {co_authorship} | keyword: {keyword}')
    print(f'name: {name} | inst1: {inst1} | title1: {title1} | site1: {site1}')
    print(f'name: {name2} | inst2: {inst2} | title2: {title2} | site1: {site2}')
    print(f'inst: {inst} | journal1: {journal1} | journal2: {journal2}')
    print('------------------------------------------------------------구분선------------------------------------------------------------')
    
    return weight

In [41]:
def getRaw2(name):
    if 'raws' not in Answer_dict[name]:
        raws = []
        for site_one in site:
            if site_one in Answer_dict[name]:
                for c in raw_dbs[site_one].find({"_id": {"$in": Answer_dict[name][site_one]['papers']}}):
                    c['site'] = site_one
                    raws.append(c)
        
        Answer_dict[name]['raws'] = raws

In [42]:
pair_test = ['biswasrajib', 'biswasr']
  
getRaw2(pair_test[0])
getRaw2(pair_test[1])

raws1 = Answer_dict[pair_test[0]]['raws']
raws2 = Answer_dict[pair_test[1]]['raws']

name1 = Answer_dict[pair_test[0]]['name']
name2 = Answer_dict[pair_test[1]]['name']

for ra1, ra2 in zip(raws1, raws2):
    site1 = ra1['site']
    site2 = ra2['site']
    inst1 = Answer_dict[pair_test[0]][site1]['oriInst']
    inst2 = Answer_dict[pair_test[1]][site2]['oriInst']
    Secondary_filter2(name1, name2, site1, inst1, ra1, site2, inst2, ra2)

weigth: 4 | joc: 0 | yop: 2021, 2021 | co_author_count: 0| keyword: [['T']], [['T']]
name: Biswas, Rajib | inst1: Department of Physics Tezpur University Tezpur 784028 India | title1: Outlining Big Data Analytics in Health Sector with Special Reference to Covid-19 | site1: SCOPUS
name: Biswas, R. | inst2: Tezpur Univ, Dept Phys, Tezpur 784028, Assam, India | title2: Outlining Big Data Analytics in Health Sector with Special Reference to Covid-19 | site1: WOS
inst: 0.7179874517860559 | journal1: Springer | journal2: WIRELESS PERS COMMUN
------------------------------------------------------------구분선2------------------------------------------------------------


In [55]:
pair_test = ['shoshaniarie', 'shoshania'] 
pair_test = ['gagliardopablo', 'gagliardop'] 
pair_test = ['huangqian', 'huangq'] 
pair_test = ['venegasmartinezfrancisco', 'venegasmartínezf'] 
pair_test = ['alzahranifahad', 'alzahranifahda']
pair_test = ['sim´eonjérôme', 'simeonjerome']
pair_test = ['grothpault', 'grothpaul']
pair_test = ['szymańskił', 'szymanskil']
pair_test = ['burinskienėaurelija', 'burinskieneaurelija']
pair_test = ['herrerafrancisco_0', 'herrerafrancisco_1'] 
pair_test = ['pakzadshamimn_0', 'pakzadshamimn_2'] 
pair_test = ['chungyehching_0', 'chungyehching_1']
pair_test = ['huangmaolin_0', 'huangmaolin_1']
pair_test = ['dhavachelvanp_0', 'dhavachelvanp_1']
pair_test = ['bajwahassan_0', 'bajwahassan_1']
pair_test = ['gugulothunarsimha_0', 'gugulothunarsimha_1']
pair_test = ['kimjeongjoon_0', 'kimjeongjoon_1']
pair_test = ['elsayedayman_0', 'elsayedayman_1']
pair_test = ['heshunmin_0', 'heshunmin_1']
pair_test = ['arunachalamar_0', 'arunachalamar_1']
pair_test = ['sharmasanjiv_0', 'sharmasanjiv_1']
pair_test = ['bouamranekarim_0', 'bouamranekarim_2']
pair_test = ['jeyanthin_0', 'jeyanthin_2']
pair_test = ['liguoming', 'ligm']
pair_test = ['chiucarter', 'chiuc'] 
pair_test = ['kwonjiwoong', 'kwonjw'] 
pair_test = ['stepanovalexanderv', 'stepanovav'] 
pair_test = ['balasubramanianmahendran', 'balasubramanianm'] 
pair_test = ['wittebolexavier', 'wittebolex'] 
pair_test = ['nikolaevichtisenkovictor', 'nikolaevichtv'] 
pair_test = ['elsoudanimagdyms', 'elsoudanimms'] 
pair_test = ['chenwei_0', 'chenwei_2']
pair_test = ['sokolenkoliudmyla', 'sokolenkol']
pair_test = ['igarashiyasuhiko', 'igarashiy']
pair_test = ['youkhyun', 'youkh']
pair_test = ['tomásjesúsgarcía', 'tomasjesus']
pair_test = ['rajcpethuru', 'rajpethuru']
pair_test = ['cristeavalentingabriel', 'cristeavalentin']
pair_test = ['sáfilipe', 'safilipe']
pair_test = ['gräbelstefan', 'graebelstefan']
pair_test = ['ionescusorincristian', 'ionescusorin']
pair_test = ['kaliappanm', 'kahappanm']
pair_test = ['jadiamrmohsen', 'jadiamr']
pair_test = ['rashidmohammadanmir', 'rashidmohammeda']
pair_test = ['biswasrajib', 'biswasr']

